In [35]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt

xlm = ET.parse('test.xml')
tag_root = xlm.getroot()
for tag in tag_root:
    print(tag)

<Element '{urn:schemas-microsoft-com:office:office}DocumentProperties' at 0x000001AC068B8860>
<Element '{urn:schemas-microsoft-com:office:excel}ExcelWorkbook' at 0x000001AC068B8590>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Styles' at 0x000001AC068B8270>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet' at 0x000001AC068A2F90>


In [28]:

level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

# df = pd.DataFrame({"Date":[], "Time":[], "Name":[], "Amount":[], "Balance":[]})

rows = []#{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[]}

flag1_mark = False  # Найти текст 'Карта'
flag2_mark = False  # Найти метку карты, найти идентификатор метки базе данных
flag3 = False  # Найти текст 'Карта'
flag_break = False
for i_row, tag_row in enumerate(tag_root.findall(level)):
    if i_row == 9:
        amounts = []
        for i, t1 in enumerate(tag_row.itertext()):
            if i >= 4: break            
            t2 = str.replace(t1, ' ', '')
            t3 = str.replace(t2, ',', '.')
            try: amount = float(t3)
            except: amount = None
            amounts.append(amount)
            
        print(amounts)
    elif i_row >= 19:        
        # print('row ', i_row, '-----------------', tag_row)

        row = []
        for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
            if i_cel > 7: break
            text = ''
            for t in tag.itertext():
                text = t
            row.append(text)
        # row.insert(0, len(row))
        if len(row) == 8:
            rows.append(row)

    

    # # print('---------------------------')
    # if flag2_mark:
    #     print('row ', i_row, '-----------------', tag_row)
    #     # for tag in tag_row:
    #         # print(tag, tag.text)
    #     for text in tag_row.itertext():
    #         print(text)
    # else:
    #     if flag1_mark:            
    #         # Найти метку карты, найти идентификатор метки базе данных
    #         # texts = tag_row.itertext()

    #         flag_break = True
    #         for i, text in  enumerate(tag_row.itertext()):
    #             # Получить ID из БД
    #             card_mark = text
    #             print(i, text)
    #             break
    #         flag2_mark = True
    #         flag_break = False

    #     else:
    #         for text in tag_row.itertext():
    #             if text == 'Карта':
    #                 flag1_mark = True
    #                 break
    # if flag_break:
    #     break


    # print(text)

#     print(tag, tag.text)
#     for tag2 in tag:
#         print('  - ', tag2, tag2.text)


[3432.97, 337.94, 129334.64, 126239.61]


In [41]:
import my_methods as mm

# for row in rows:
#     print(row)
columns=["Date", "Time", "ccccc", "Name", "zzzzz", "Amount", "fffff", "Balance"]
df = pd.DataFrame(columns=columns, data=rows)
df["Amount_num"] = df.Amount.apply(mm.text_to_float)
df["Balance_num"] = df.Balance.apply(mm.text_to_float)
df.to_csv('test_out1.csv', sep=';', decimal=',', encoding='cp1251')

len1 = len(rows)
if len1%2 != 0:
    error_text = f'Не четное количество строк в таблице: {len1}'
    print(error_text)

#   Date	    Time  ...   Name	...	         Amount	....	Balance	Amount_num	Balance_num
# 0	2022-03-31	15:44		Перевод с карты		5 000,00		337.94	5000.00	337.94
# 1	2022-03-31	202880		SBOL перевод 2202****6497 В. СЕРГЕЙ СЕРГЕЕВИЧ					NaN	NaN

rows2 = []
for i1 in range(0, len1, 2):
    r1 = rows[i1]
    r2 = rows[i1+1]    
    row = []
    
    # date = DateTimeField()
    val = dt.datetime.strptime(r1[0] + ' ' + r1[1], '%Y-%m-%d %H:%M')
    row.append(val)

    # date_account = DateTimeField(null=True)    
    val = dt.datetime.strptime(r2[0], '%Y-%m-%d')
    row.append(val)

    # category = CharField(null=True) 
    val = r1[3]
    row.append(val)

    # name = CharField(null=True)
    val = r2[3]
    row.append(val)

    # amount = FloatField()
    st = r1[5]
    val = mm.text_to_float(st) if st[0] == '+' else -mm.text_to_float(st)
    row.append(val)

    # balance
    val = mm.text_to_float(r1[7])
    row.append(val)

    # Сохранить в список 
    rows2.append(row)

columns2=["date", "date_account", "category", "name", "amount", "balance"]
df = pd.DataFrame(columns=columns2, data=rows2).sort_values(by='date')
df.to_csv('test_out2.csv', sep=';', decimal=',', encoding='cp1251')







